# Table of Contents:
* [Introduction](#intro)
    * [Data](#data)
        *  [Analytical Process](#ap)
            * [Successful vs. Failed](#svf)
            * [Defining a Successful Campaign](#dsc)
    * [Final Recommendations](#finalrec)

# Introduction <a class="anchor" id="intro"></a>

Nowadays there are various methods in receiving additional funding/capital for company projects, one being Kickstarter. In short, Kickstarter is an online funding platform allowing anyone to back up projects from all over the world at any given time. It is important to understand what makes a campaign succeed on Kickstarter and so, data analysis has been conducted to aid in the decision-making process of future campaigns.

## Data <a class="anchor" id="data"></a>

The data used in analysis is composed of 1500 Kickstarter campaigns with several attributes such as campaign goal, money raised, campaign launch date, campaign end date, number of backers, and if the campaign was successful. Below is a sample of the first row in the campaigns table with the number of campaigns as the last column.

In [6]:
# Load SQLlite & DB file 
%load_ext sql
con = "sqlite:////Users/jadel/Desktop/kickstarter/kickstarter.db"
# Or you can connect directly like this:
# %sql "sqlite:////Users/jadel/Desktop/kickstarter/kickstarter.db"

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [7]:
%%sql $con

SELECT *, COUNT(id)
FROM campaign

Done.


id,name,sub_category_id,country_id,currency_id,launched,deadline,goal,pledged,backers,outcome,COUNT(id)
1,Ragdolls,23,2,2,2013-04-25 00:00:00,2013-05-25 00:00:00,15000,20,3,failed,15000


### Analytical Process <a class="anchor" id="ap"></a>

#### Successful vs Failed <a class="anchor" id="svf"></a>

The first step in the analytical process is to find the main drivers between a failed and successful campaign. We start with taking a look at the "Average Goal Price USD" and the "Average Pledged Price USD" on both outcomes (Failed vs Successful) to get an idea if there is a significant difference between the two.

In [8]:
%%sql $con         /*Failed Campaigns*/

SELECT ROUND(AVG(goal)) AS Average_Goal_Price, 
	   ROUND(AVG(pledged)) AS Average_Pledged_Price,
       outcome
FROM campaign
WHERE currency_id = 2 AND outcome = "failed" 

UNION 

              /*Successful Campaigns*/

SELECT ROUND(AVG(goal)) AS Average_Goal_Price,
	   ROUND(AVG(pledged)) AS Average_Pledged_Price, 
       outcome
FROM campaign
WHERE currency_id = 2 AND outcome = "successful"; 



Done.


Average_Goal_Price,Average_Pledged_Price,outcome
9976.0,23117.0,successful
101653.0,1528.0,failed


As seen above, there is a significant difference between successful and unsuccessful campaign goals. On average, failed campaigns seem to request for too much money, <span>$</span>101653, and in return receive around 1.5% back, $1528. Meanwhile, successful campaigns on average ask for a much lower amount, $9976, and in response receive over double their goal, $23117.

What this shows is that very large campaign goals are not attainable and likely are fending off potential backers. A large part of Kickstarter is the concept of supporting creative ideas and a backer can feel his/her impact on a campaign when the goal is low enough for a single donation to make a visible difference. To further clarify, if a backer has $500 to support any campaign and has a choice between a $100K or a $10K campaign goal the backer knows that its $500 will make a larger impact on the lower end goal. In percentages, it’s a choice between a 5% or 0.5% impact. Please see below for a visualization summary

![Alt text](Images/average_campaign_funding.png)

Additionally, there is a very slight increase in money raised for longer campaigns, this can be seen visually below where the trend line minorly increases as a campaign is on for longer. This suggests that campaigns have a life cycle and at some point, are considered “old news” as other newer campaigns take the spotlight on Kickstarter. Overall, extending a campaign does not seem to hurt nor benefit a goal significantly but rather keep it stagnant.

In [9]:
%%sql $con	
				/* Pledges vs. Duration in Days */
SELECT ROUND(pledged) AS money_raised, 
		JULIANDAY(deadline) - JULIANDAY(launched) AS duration_in_days
FROM campaign
ORDER BY pledged desc
LIMIT 10;

Done.


money_raised,duration_in_days
5408917.0,35.0
3999796.0,28.0
3105473.0,30.0
2278255.0,30.0
1924018.0,30.0
1842142.0,43.0
1652247.0,40.0
1546270.0,30.0
1341305.0,33.0
1241615.0,45.0


![Alt text](Images/trend_in_money_raising.png)

#### Defining a Successful Campaign <a class="anchor" id="dsc"></a>

Once campaigns can be differentiated based on their outcome, it is important to see the characteristics of just successful campaigns to determine what defines a working campaign. 

First, looking at the number of backers in a campaign can signify what is popular and trendy. 

In [10]:
%%sql $con
                /* Top Backed Categories*/
SELECT SUM(campaign.backers) AS backers, category.name AS Category
FROM campaign
JOIN sub_category
ON campaign.sub_category_id = sub_category.id
JOIN category
ON sub_category.category_id = category.id
GROUP BY category_id
ORDER BY backers desc
LIMIT 3

Done.


backers,Category
411671,Games
329751,Technology
262245,Design


In [11]:
%%sql $con
                    /* Top Backed Subcategories*/
SELECT SUM(campaign.backers) AS backers, sub_category.name AS Subcategory 
FROM campaign
JOIN sub_category
ON campaign.sub_category_id = sub_category.id
JOIN category
ON sub_category.category_id = category.id
GROUP BY sub_category_id
ORDER BY backers desc
LIMIT 3

Done.


backers,Subcategory
247120,Tabletop Games
221931,Product Design
141052,Video Games


The most backed campaign categories are Games, Technology, Design. Additionally, the most backed subcategories include Tabletop Games, Product Design, Video Games.

Having a business fall under the top 3 most backed sub/categories may come with barriers of entry and higher competition but also includes the potential of virality. A bar graph representation of most backed categories and subcategories is shown below

![Alt text](Images/campaign_categories_ranked_by_backers.png) 
![Alt text](Images/campaign_subcategories_ranked_by_backers.png)



Second, the variable "pledged" indicates the amount of money raised by a campaign. This is an important trait to define successful campaigns with. 

In [12]:
%%sql $con
                        /* Top Pledged Categories*/
SELECT ROUND(SUM(campaign.pledged)) AS pledged, category.name AS Category
FROM campaign
JOIN sub_category
ON campaign.sub_category_id = sub_category.id
JOIN category
ON sub_category.category_id = category.id
GROUP BY category.id
ORDER BY pledged desc
LIMIT 3

Done.


pledged,Category
28035408.0,Technology
27792570.0,Games
24006972.0,Design


In [13]:
%%sql $con
                            /* Top Pledged Subcategories*/
SELECT ROUND(SUM(campaign.pledged)) AS pledged, sub_category.name AS Subcategory
FROM campaign
JOIN sub_category
ON campaign.sub_category_id = sub_category.id
JOIN category
ON sub_category.category_id = category.id
GROUP BY sub_category_id
ORDER BY pledged desc
LIMIT 3

Done.


pledged,Subcategory
21111582.0,Product Design
18827697.0,Tabletop Games
7811751.0,Video Games


One of the highest monies earning subcategories include Tabletop Games. In fact, the campaign Gloomhaven Second Printing has raised around 4 million dollars in approximately a 19-million-dollar subcategory, that is around 21% of all the money raised in the Tabletop Games subcategory! Gloomhaven’s success is proof that board game companies can utilize their popularity on Kickstarter’s platform to achieve over and above their campaign goal. More detail on campaign rankings based on money raised can be found in the visualization below.

In [14]:
%%sql $con
                        /* Top Board Game Company */
SELECT name, ROUND(pledged) AS money_raised, backers, 
		(SELECT ROUND(SUM(pledged))
			FROM campaign
			WHERE sub_category_id = 14
			ORDER BY pledged desc) AS total_money_raised,
		ROUND(pledged)/(SELECT ROUND(SUM(pledged))
			FROM campaign
			WHERE sub_category_id = 14
			ORDER BY pledged desc) AS ratio_money_raised
FROM campaign
WHERE sub_category_id = 14
ORDER BY pledged desc
LIMIT 1

Done.


name,money_raised,backers,total_money_raised,ratio_money_raised
Gloomhaven (Second Printing),3999796.0,40642,18827697.0,0.21244212714916752


![Alt text](Images/campaign_categories_ranked_by_money_raised.png)
![Alt text](Images/campaign_subcategories_ranked_by_money_raised.png)

Finally, categorizing money raised and backers by country can show target locations to attract as many backers and money as possible. Top locations in this regard fall to US, Great Britain, and Canada. For more details and a map visualization see below.

In [15]:
%%sql $con
                    /* Top 3 Pledged Countries*/
SELECT ROUND(SUM(campaign.pledged)) AS money_raised, country.name AS country
FROM campaign
JOIN country
ON campaign.country_id = country.id
WHERE outcome = "successful"
GROUP BY country
ORDER BY money_raised desc
LIMIT 3

Done.


money_raised,country
100971994.0,US
8514817.0,GB
1804147.0,CA


In [16]:
%%sql $con
                    /* Top 3 Backed Countries*/
SELECT SUM(campaign.backers) AS backers, country.name AS country
FROM campaign
JOIN country
ON campaign.country_id = country.id
WHERE outcome = "successful"
GROUP BY country
ORDER BY backers desc
LIMIT 3

Done.


backers,country
1295509,US
90729,GB
29704,AU


<img src = Images/countries_map.png\
svg.png width = "1000" height = "542" >


## Final Recomendations <a class="anchor" id="finalrec"></a>

Firstly, based on the Kickstarter data it is recommended to pursue a campaign in the top backed sub/categories due to the advantageous position that they hold. For instance, the Tabletop subcategory is one of the most popular amongst backers on Kickstarter. Moreover, an optimal goal price would be towards the lower end matching the findings done above, where a lower campaign goal is likely more successful. About 47% of successful Tabletop campaigns have a goal price of \$20K or less, setting a higher goal such as \$30K only increases the number of successes by around 3% then barely increases as the goal goes higher and higher. This suggests that the analysis done on the "Trend in Money Raising" is valid, and a higher campaign goal yields a minimal benefit. Thus, it is highly advised to set a campaign goal of \$20K. 

In [17]:
%%sql $con
                
                            /* Expected Campaign Goal */
/* Use atleast one decimal to tell SQL lite that you want decimals when running arithmetic operations */

SELECT
		SUM(CASE
				WHEN sub_category_id = 14 AND goal <= 20000 AND outcome = "successful"
				THEN 1 
				ELSE 0 END) AS tabletop, 
		SUM(CASE
				WHEN sub_category_id = 14 
				THEN 1 
				ELSE 0 END) AS total_tabletop,
		ROUND(
            (SUM(CASE
				WHEN sub_category_id = 14 AND goal <= 20000 AND outcome = "successful"
				THEN 1.0    
				ELSE 0 END) 
                / 
		    SUM(CASE
				WHEN sub_category_id = 14 
				THEN 1 
				ELSE 0 END)) * 100) AS ratio
FROM campaign


Done.


tabletop,total_tabletop,ratio
261,553,47.0


Secondly, the average Kickstarter Backer is willing to pledge \$80 and the expected number of backers to support a Tabletop campaign of \$20K is 282. Therefore, the total expected money raised of \$22,560 exceeds the campaign goal of $20K. To conclude, final recommendations for a Tabletop campaign is to set a \$20K goal with the expectation of surpassing it with the help of around 282 backers.

In [18]:
%%sql $con
                    /* Average Amount a Backer Pledges on Kickstarter */
SELECT ROUND(AVG(backers)) AS expected_backers, 

		(SELECT ROUND(AVG(pledged) / AVG(backers))
        FROM campaign) AS amount_per_backer, 
        
        (SELECT (SELECT ROUND(AVG(backers))) * 
        (SELECT ROUND(AVG(pledged) / AVG(backers))
        FROM campaign)) AS expected_money_raised

FROM campaign
WHERE sub_category_id = 14 AND goal <= 20000

Done.


expected_backers,amount_per_backer,expected_money_raised
282.0,80.0,22560.0
